In [3]:
%matplotlib inline
import pandas as pd
import matplotlib
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Average window_stride elements together to form a single row
window_stride = 12

sample_hours = window_stride / 12.0
print("Sample Hours: %f" % sample_hours)

# Number of future samples to mean for prediction
prediction_window = int(24 / sample_hours)
print("Prediction Window: %d" % prediction_window)

# Length of the windowed sequence
sequence_length = int(7*24 / sample_hours)
print("Sequence Length: %d" % sequence_length)

Sample Hours: 1.000000
Prediction Window: 24
Sequence Length: 168


In [4]:
# Read the data
df = pd.read_csv('D:/programming-no-gdrive/air-pollution/data-formatted/ready/Data_2000_ready.csv')

In [7]:
df.columns

Index(['AQS_Code', 'Latitude', 'Longitude', 'epoch', 'no', 'no2', 'nox', 'o3',
       'temp', 'temp_flag', 'winddir', 'windspd', 'hour', 'day', 'month',
       'year', 'wind_x_dir', 'wind_y_dir'],
      dtype='object')

In [8]:
#formatting if needed
df['month']=df['month'].apply(lambda x: str(x))
df['hour']=df['hour'].apply(lambda x: str(x))
df['day']=df['day'].apply(lambda x: str(x))
pre_dummies=pd.concat([df['hour'], df['day'], df['month']], axis=1)
dummies=pd.get_dummies(pre_dummies)
df=pd.concat([df, dummies], axis=1)
df=df.drop_duplicates()
df=df.drop(['hour', 'day', 'month', 'temp_flag'], axis=1)

In [14]:
for column in df.columns:
    print(column)
    print(df[column].dtype)

AQS_Code
object
Latitude
float64
Longitude
float64
epoch
int64
no
float64
no2
float64
nox
float64
o3
float64
temp
float64
winddir
float64
windspd
float64
year
int64
wind_x_dir
float64
wind_y_dir
float64
hour_0
uint8
hour_1
uint8
hour_10
uint8
hour_11
uint8
hour_12
uint8
hour_13
uint8
hour_14
uint8
hour_15
uint8
hour_16
uint8
hour_17
uint8
hour_18
uint8
hour_19
uint8
hour_2
uint8
hour_20
uint8
hour_21
uint8
hour_22
uint8
hour_23
uint8
hour_3
uint8
hour_4
uint8
hour_5
uint8
hour_6
uint8
hour_7
uint8
hour_8
uint8
hour_9
uint8
day_1
uint8
day_10
uint8
day_11
uint8
day_12
uint8
day_13
uint8
day_14
uint8
day_15
uint8
day_16
uint8
day_17
uint8
day_18
uint8
day_19
uint8
day_2
uint8
day_20
uint8
day_21
uint8
day_22
uint8
day_23
uint8
day_24
uint8
day_25
uint8
day_26
uint8
day_27
uint8
day_28
uint8
day_29
uint8
day_3
uint8
day_30
uint8
day_31
uint8
day_4
uint8
day_5
uint8
day_6
uint8
day_7
uint8
day_8
uint8
day_9
uint8
month_1
uint8
month_10
uint8
month_11
uint8
month_12
uint8
month_2
uint8
mont

In [21]:
# Drop bad rows
#df = df[df['val']=='y']

# Input Features
columns=df.columns

# Number of features we take from the data
input_features = len(columns)
num_features = input_features
num_inputs = input_features

# Number of things we are doing regression to predict
num_outputs = 4

# Unprocessed dataset
nd = df[columns].values

# Windowed dataset
nd_window = np.zeros((int(nd.shape[0] / window_stride), num_inputs))

row = 0
while row < nd.shape[0] - window_stride:
    for i in range(0, input_features):
        nd_window[int(row/window_stride)][i] = np.mean(nd[row:row+window_stride,i])
    row += window_stride
    
scaler = MinMaxScaler()
scaler.fit(nd_window)
nd_window = scaler.transform(nd_window)

# Create sequences
data = []
labels = []

rows = deque(maxlen=sequence_length)

for idx, r in enumerate(nd_window):

    rows.append([a for a in r])
    
    # We need the entire sequence filled to make a prediction about the future mean
    if len(rows) < sequence_length:
        continue
    
    # Since we are predicting the mean, make sure we do not go out of bounds in the future
    if idx+1 + prediction_window > nd_window.shape[0]:
        break
        
    data.append(rows.copy())
        
    # We are predicting the future mean values
    u_24_no = np.mean( nd_window[idx+1 : idx+1 + prediction_window, 4] )
    u_24_no2 = np.mean( nd_window[idx+1 : idx+1 + prediction_window, 5] )
    u_24_nox = np.mean( nd_window[idx+1 : idx+1 + prediction_window, 6] )
    u_24_o3 = np.mean( nd_window[idx+1 : idx+1 + prediction_window, 7] )
    
    labels.append([u_24_no, u_24_no2, u_24_nox, u_24_o3])

data = np.array(data)
data = data.reshape((data.shape[0], data.shape[1]*data.shape[2]))
labels = np.array(labels)

print(data.shape)
print(labels.shape)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.33, random_state=42)

0


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [5]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(random_state=0, n_estimators=100, n_jobs=-1)
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

 

0.8287807335243608

In [ ]:
plt.rcParams['figure.figsize'] = (20, 10)
plt.rcParams['font.size'] = 16

for seq in range(0, data.shape[0] - sequence_length):
    
    lookup = {'no': (0, 0), 'no2':(0, 1), 'nox':(1, 0), 'o3':(1, 1)}

    pred = regr.predict(data[seq].reshape((1, 1512)))[0]
    fig, ax = plt.subplots(2, 2)

    for idx,f in enumerate([(4, 'no'), (5, 'no2'), (6, 'nox'), (7, 'o3')]):
    
        feature_index, feature_name = f
        
        X = []
        Y_actual = []

        for i in range(0, sequence_length + int(24*(1/sample_hours))):
            X.append(seq+i)
            Y_actual.append(data[seq+i][feature_index])

        Y_actual = np.array(Y_actual)
        
        predicted_mean = pred[feature_index - 4]
        actual_mean = np.mean(Y_actual[sequence_length:])
        rolling_mean = np.mean(Y_actual[:sequence_length])
        rolling_std = np.std(Y_actual[:sequence_length])
                
        Y_pred = Y_actual.copy()
        Y_pred[sequence_length:] = predicted_mean
        Y_pred[:sequence_length] = np.nan

        Y_actual_mean = Y_actual.copy()
        Y_actual_mean[sequence_length:] = actual_mean
        Y_actual_mean[:sequence_length] = np.nan
        
        Y_rolling_mean = Y_actual.copy()
        Y_rolling_mean[:sequence_length] = rolling_mean
        Y_rolling_mean[sequence_length:] = np.nan
        
        Y_rolling_std_upper = Y_actual.copy()
        Y_rolling_std_upper[:sequence_length] = rolling_mean + rolling_std
        Y_rolling_std_upper[sequence_length:] = np.nan
        
        Y_rolling_std_lower = Y_actual.copy()
        Y_rolling_std_lower[:sequence_length] = rolling_mean - rolling_std
        Y_rolling_std_lower[sequence_length:] = np.nan   
        
        subplot = ax[lookup[feature_name][0]][lookup[feature_name][1]]

        subplot.plot(X, Y_actual, color='black', linewidth=4.0)
        subplot.plot(X, Y_actual_mean, color='green', linewidth=4.0)
        subplot.plot(X, Y_pred, color='purple', linewidth=4.0)
        subplot.plot(X, Y_rolling_mean, color='green', linewidth=4.0)
        subplot.plot(X, Y_rolling_std_upper, color='orange', linewidth=4.0)
        subplot.plot(X, Y_rolling_std_lower, color='orange', linewidth=4.0)
        
        subplot.grid()
        
        subplot.set_title("%s 24 hour mean prediction" % (feature_name,))
        
        subplot.set_xlabel("Hours")
        subplot.set_ylabel("Scaled Concentration")
    
    fig.legend(['Actual Continuous', 'Actual Mean', 'Predicted Mean', 'Rolling Mean', 'Standard Deviation'])
    fig.tight_layout()

    plt.savefig('charts/%.05d.png' % seq)
    # plt.show()
    plt.close()

    print("Rendered %d" % seq)

Rendered 0
Rendered 1
Rendered 2
Rendered 3
Rendered 4
Rendered 5
Rendered 6
Rendered 7
Rendered 8
Rendered 9
Rendered 10
Rendered 11
Rendered 12
Rendered 13
Rendered 14
Rendered 15
Rendered 16
Rendered 17
Rendered 18
Rendered 19
Rendered 20
Rendered 21
Rendered 22
Rendered 23
Rendered 24
Rendered 25
Rendered 26
Rendered 27
Rendered 28
Rendered 29
Rendered 30
Rendered 31
Rendered 32
Rendered 33
Rendered 34
Rendered 35
Rendered 36
Rendered 37
Rendered 38
Rendered 39
Rendered 40
Rendered 41
Rendered 42
Rendered 43
Rendered 44
Rendered 45
Rendered 46
Rendered 47
Rendered 48
Rendered 49
Rendered 50
Rendered 51
Rendered 52
Rendered 53
Rendered 54
Rendered 55
Rendered 56
Rendered 57
Rendered 58
Rendered 59
Rendered 60
Rendered 61
Rendered 62
Rendered 63
Rendered 64
Rendered 65
Rendered 66
Rendered 67
Rendered 68
Rendered 69
Rendered 70
Rendered 71
Rendered 72
Rendered 73
Rendered 74
Rendered 75
Rendered 76
Rendered 77
Rendered 78
Rendered 79
Rendered 80
Rendered 81
Rendered 82
Rendered 83
Re

In [29]:
name = {}

f = 0
for s in range(0, sequence_length):
    key = s - (sequence_length-1)
    name[f+0] = "%d_hour" % key
    name[f+1] = "%d_temp" % key
    name[f+2] = "%d_windspd" % key
    name[f+3] = "%d_winddir" % key
    name[f+4] = "%d_no" % key
    name[f+5] = "%d_no2" % key
    name[f+6] = "%d_nox" % key
    name[f+7] = "%d_o3" % key
    
    f += 8
    
    if fft_features:
        for i in range(0, 8):
            for r in range(0, 12):
                name[f] = "%d_fft_%d_%d" % (key, i, r)
                f += 1

pairs = []

for idx, imp in enumerate(regr.feature_importances_):    
    pairs.append([imp, name[idx]])
    
    
pairs.sort(reverse=True)
for v in pairs:
    value, key = v
    print("%s:\t\t\t%f" % (key, value))

0_temp:			0.188156
0_no2:			0.094603
0_o3:			0.037851
0_nox:			0.026878
-1_temp:			0.022386
-10_temp:			0.008741
-2_no2:			0.008526
-2_temp:			0.008267
-1_no2:			0.007111
-144_temp:			0.006583
-13_o3:			0.006280
-1_o3:			0.006097
-143_temp:			0.005876
-3_temp:			0.005185
-12_temp:			0.005035
0_winddir:			0.004986
-12_o3:			0.004693
-11_temp:			0.004454
-9_temp:			0.004248
-145_temp:			0.004056
-2_winddir:			0.003471
-50_no2:			0.002974
-6_temp:			0.002960
-57_temp:			0.002881
-13_temp:			0.002808
-4_temp:			0.002747
-11_o3:			0.002600
-4_winddir:			0.002571
-167_temp:			0.002507
-3_winddir:			0.002443
-38_winddir:			0.002423
-1_winddir:			0.002362
-8_winddir:			0.002296
-9_o3:			0.002112
-2_nox:			0.002065
-8_o3:			0.002061
-5_winddir:			0.002037
-14_o3:			0.002004
-149_temp:			0.001950
-142_temp:			0.001899
0_windspd:			0.001768
-7_o3:			0.001711
-6_winddir:			0.001666
-98_no2:			0.001647
-9_winddir:			0.001619
-5_no:			0.001610
-97_temp:			0.001535
-147_temp:			0.001533
-10_winddir:	